Installs

In [ ]:
pip install tokenizers

     |████████████████████████████████| 3.0MB 9.0MB/s 


In [ ]:
pip install transformers

     |████████████████████████████████| 778kB 8.7MB/s 
     |████████████████████████████████| 890kB 28.2MB/s 
     |████████████████████████████████| 1.1MB 39.8MB/s 
     |████████████████████████████████| 3.0MB 46.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=1ca378681e5aa785fdf5c9a1596a394312531080049c9237fd4cf8e7dc33de4b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
  Found existing installation: tokenizers 0.8.1
    Uninstalling tokenizers-0.8.1:
      Successfully uninstalled tokenizers-0.8.1


Imports

In [ ]:
import os
import torch
import numpy as np
import pandas as pd
from transformers import BertConfig, BertForMaskedLM, BertModel
from transformers import Trainer, TrainingArguments
from tokenizers import BertWordPieceTokenizer

In [ ]:
from _dataset import BERT16SDataset
from _collator import DataCollatorForBertWordPieceTokenizer

Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
drive_path = './drive/My Drive/Colab Notebooks/NLP/model'

Check Resources

In [ ]:
# Check that we have a GPU
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [ ]:
# Check that PyTorch sees it
torch.cuda.is_available()

True

Prepare Model Config

In [ ]:
vocab_size = 15621  # parallel to k=6 in classic k-mers (for this corpus)

In [ ]:
config = BertConfig(
    vocab_size=vocab_size,
    hidden_size=256,
    intermediate_size=1024,
    num_hidden_layers=4,
    num_attention_heads=4,
    max_position_embeddings=512
)

Create BERT model

In [ ]:
model = BertForMaskedLM(config=config)

print(f"BERT model has {model.num_parameters()/10**6}M parameters")

BERT model has 7.437829M parameters


Create Datatset

In [ ]:
vocab_path = os.path.join(drive_path, 'vocab.txt')
data_path = os.path.join(drive_path, 'SILVA_parsed_V2.tsv')

In [ ]:
dataset = BERT16SDataset(
    vocab_path=vocab_path,
    data_path=data_path,
    block_size=512
)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Crete Data Collator

In [ ]:
tokenizer = BertWordPieceTokenizer(
    vocab_path,
    handle_chinese_chars=False,
    lowercase=False,
    unk_token="[UNK]",
    sep_token="[SEP]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    mask_token="[MASK]")

tokenizer.enable_truncation(512)
tokenizer.enable_padding(length=512)

In [ ]:
len(tokenizer.get_vocab())

15621

In [ ]:
data_collator = DataCollatorForBertWordPieceTokenizer(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
model(data_collator.collate_batch([dataset[0]])['input_ids'])[0].shape

torch.Size([1, 512, 15621])

Config Training

In [ ]:
#model = BertForMaskedLM.from_pretrained(drive_path)

In [ ]:
training_args = TrainingArguments(
    output_dir=os.path.join(drive_path, 'checkpoints'),
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=40,
    save_steps=10_000,
    logging_steps=500,
    save_total_limit=2,
    learning_rate=5e-4,
)

In [ ]:
from typing import Callable, Dict, List, Optional, Tuple
import json
from transformers.optimization import get_constant_schedule_with_warmup

class CustomTrainer(Trainer):

    def _log(self, logs: Dict[str, float], iterator: Optional = None) -> None:
      if self.epoch is not None:
          logs["epoch"] = self.epoch

      output = json.dumps({**logs, **{"step": self.global_step}})
      print(output)
  
    def get_optimizers(self, num_training_steps: int) -> Tuple[torch.optim.Optimizer, torch.optim.lr_scheduler.LambdaLR]:
        optimizer, _ = super(CustomTrainer, self).get_optimizers(num_training_steps)
        scheduler = get_constant_schedule_with_warmup(optimizer, num_warmup_steps=self.args.warmup_steps)
        return optimizer, scheduler

In [ ]:
trainer = CustomTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
    prediction_loss_only=True,
)

/usr/local/lib/python3.6/dist-packages/transformers/trainer.py:226: FutureWarning: The `data_collator` should now be a simple callable (function, class with `__call__`), classes with a `collate_batch` are deprecated and won't be supported in a future version.
  FutureWarning,


Train!

In [ ]:
%%time
trainer.train()

{"loss": 2.1806337041854857, "learning_rate": 0.0005, "epoch": 0.04629200999907416, "step": 500}
{"loss": 2.178268550634384, "learning_rate": 0.0005, "epoch": 0.09258401999814832, "step": 1000}
{"loss": 2.161776591539383, "learning_rate": 0.0005, "epoch": 0.13887602999722248, "step": 1500}
{"loss": 2.166535542011261, "learning_rate": 0.0005, "epoch": 0.18516803999629664, "step": 2000}
{"loss": 2.1483408017158507, "learning_rate": 0.0005, "epoch": 0.2314600499953708, "step": 2500}
{"loss": 2.1512983677387236, "learning_rate": 0.0005, "epoch": 0.27775205999444497, "step": 3000}
{"loss": 2.14949022769928, "learning_rate": 0.0005, "epoch": 0.32404406999351915, "step": 3500}
{"loss": 2.149521510362625, "learning_rate": 0.0005, "epoch": 0.37033607999259327, "step": 4000}
{"loss": 2.142380564212799, "learning_rate": 0.0005, "epoch": 0.41662808999166745, "step": 4500}
{"loss": 2.139502103805542, "learning_rate": 0.0005, "epoch": 0.4629200999907416, "step": 5000}
{"loss": 2.1218170828819276, "l

FileNotFoundError: ignored

In [ ]:
trainer.save_model(drive_path)

In [ ]:
tokenizer.save_model(drive_path)

['./drive/My Drive/Colab Notebooks/NLP/model/vocab.txt']

Extract Embeddings

In [ ]:
loaded_model = BertModel.from_pretrained(drive_path)

In [ ]:
batch_size = 32
dataloader = torch.utils.data.DataLoader(
                dataset=dataset,
                batch_size=batch_size,
                sampler=torch.utils.data.SequentialSampler(dataset),
                num_workers=0,
                pin_memory=True
)

In [ ]:
from tqdm import tqdm

In [ ]:
%%time 

averaged_embeddings = torch.tensor([], dtype=torch.float)
first_batch = True

for batch in tqdm(dataloader, position=0, leave=True):
    model_outputs = loaded_model.embeddings(batch).mean(dim=1)
    if first_batch:
      assert batch.shape == torch.Size([batch_size, 512])
      assert model_outputs.shape == torch.Size([batch_size, 256])
      first_batch = False

    averaged_embeddings = torch.cat((averaged_embeddings, model_outputs.detach().cpu()), 0)

100%|██████████| 13502/13502 [40:43<00:00,  5.53it/s]

CPU times: user 39min 38s, sys: 1min 6s, total: 40min 44s
Wall time: 40min 43s


In [ ]:
torch.save(averaged_embeddings, os.path.join(drive_path, 'averaged_embeddings'))

In [ ]:
assert averaged_embeddings.shape[0] == len(dataset)

Extract Weighted Embedding Without Token Padding

In [ ]:
token_weights_df = pd.read_csv(os.path.join(drive_path, 'bpe_token_weights.tsv'), sep='\t')
token_weights = token_weights_df.set_index('token')['weight2'].to_dict()

In [ ]:
unpadded_tokenizer = BertWordPieceTokenizer(
    vocab_path,
    handle_chinese_chars=False,
    lowercase=False,
    unk_token="[UNK]",
    sep_token="[SEP]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    mask_token="[MASK]")

In [ ]:
averaged_embeddings = torch.tensor([], dtype=torch.float)

for sample in tqdm(dataset.samples[:50000], position=0, leave=True):

  sample_splitted = dataset._split_sequence_by_max_word_length(sample)
  tokens = dataset.tokenizer.encode(sample_splitted)
  embedding = loaded_model.embeddings(torch.tensor(tokens.ids, dtype=torch.long).expand(1, -1))

  tokens_unpadded = unpadded_tokenizer.encode(sample_splitted, add_special_tokens=False)
  original_len = len(tokens_unpadded.ids)

  weights = np.array([token_weights[k] for k in tokens_unpadded.tokens])
  weights_normalized = weights# / np.sum(weights)
  weights_tensor = torch.tensor([w * np.ones([256]) for w in weights_normalized]).view(1, -1, 256)

  averaged_embedding = torch.mul(embedding[:, 1:original_len + 1, :], weights_tensor).mean(dim=1)
  averaged_embeddings = torch.cat((averaged_embeddings, averaged_embedding.detach().cpu()), 0)

100%|██████████| 50000/50000 [21:04<00:00, 39.54it/s]


In [ ]:
torch.save(averaged_embeddings, os.path.join(drive_path, 'averaged_embeddings_no_padding__weight2'))

In [ ]:
assert averaged_embeddings.shape[0] == len(dataset)

AssertionError: ignored